In [5]:
%load_ext autoreload

%autoreload 
import os
os.chdir("..\\..\\")


In [7]:
import networkx as nx
import numpy as np
from src.visualization.graph_plot_utils import plot_graph, plot_infection

In [39]:
# Start with 1% of population infected
def initial_infection(G, p):
    for node in G.nodes():
        G.nodes[node].update({'infected' : np.random.choice(2, p = [1-p, p])})

In [46]:
G = nx.relaxed_caveman_graph(25, 4, 0.25, seed=42)
initial_infection(G, .05)

In [47]:
plot_infection(G)

In [48]:
def spread_one_step(G):
    newly_infected = []
    for node, adjacencies in enumerate(G.adjacency()):
        if G.nodes[node]['infected']: newly_infected.extend(list(adjacencies[1].keys()))

    for node in np.unique(newly_infected):
        G.nodes[node].update({'infected' : 1})

In [55]:
spread_one_step(G)
plot_infection(G)